# **Automatic Facial Blur**

In [1]:
import numpy as np
import cv2

In [3]:
# Load image
img = cv2.imread("Input/happy_parents.jpg")

cv2.imshow("original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
# Load the face detection model
face_detect_model = cv2.dnn.readNetFromCaffe("Input/models2/deploy.prototxt.txt", "Input/models2/res10_300x300_ssd_iter_140000_fp16.caffemodel")

In [23]:
image = img.copy()

# Mask image
face_mask = np.zeros(image.shape[:2], dtype="uint8")

# Blob from image
blob = cv2.dnn.blobFromImage(image, 1, (300,300), (104,177,123), swapRB=True)
face_detect_model.setInput(blob)
detect = face_detect_model.forward()

# Select faces using confidence score and get bounding box
h, w = image.shape[:2]
for i in range(detect.shape[2]):
    confidence = detect[0,0,i,2]
    if confidence > 0.5:
        box = detect[0,0,i,3:7]*np.array([w,h,w,h])
        box = box.astype(int)
        cv2.rectangle(face_mask, (box[0], box[1]), (box[2], box[3]), 255, -1)

# Inverse mask
inv_face_mask = cv2.bitwise_not(face_mask)

# Apply mask image to original image
bg_img = cv2.bitwise_and(image, image, mask=face_mask)
fg_img = cv2.bitwise_and(image, image, mask=inv_face_mask)

# Blur bg_img with Gaussian blur
bg_img_blur = cv2.GaussianBlur(bg_img, (51, 51), 0)

# Addition operation
face_blur = cv2.add(fg_img, bg_img_blur)

cv2.imshow("face original", image)
cv2.imshow("face blur", face_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
detect.shape

(1, 1, 200, 7)

In [24]:
# Create function

def facial_blur(img, ksize):

    image = img.copy()
    
    # Mask image
    face_mask = np.zeros(image.shape[:2], dtype="uint8")
    
    # Blob from image
    blob = cv2.dnn.blobFromImage(image, 1, (300,300), (104,177,123), swapRB=True)
    face_detect_model.setInput(blob)
    detect = face_detect_model.forward()
    
    # Select faces using confidence score and get bounding box
    h, w = image.shape[:2]
    for i in range(detect.shape[2]):
        confidence = detect[0,0,i,2]
        if confidence > 0.5:
            box = detect[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype(int)
            cv2.rectangle(face_mask, (box[0], box[1]), (box[2], box[3]), 255, -1)
    
    # Inverse mask
    inv_face_mask = cv2.bitwise_not(face_mask)
    
    # Apply mask image to original image
    bg_img = cv2.bitwise_and(image, image, mask=face_mask)
    fg_img = cv2.bitwise_and(image, image, mask=inv_face_mask)
    
    # Blur bg_img with Gaussian blur
    bg_img_blur = cv2.GaussianBlur(bg_img, (ksize, ksize), 0)
    
    # Addition operation
    face_blur = cv2.add(fg_img, bg_img_blur)

    return face_blur

In [26]:
faceblur = facial_blur(img, 51)

cv2.imshow("face original", image)
cv2.imshow("face blur", faceblur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
cv2.imwrite("Output/face blur.png", faceblur)

True

## Real-time Facial Blur

In [28]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if ret == False:
        break

    blur_img = facial_blur(frame, 51)

    cv2.imshow("original", frame)
    cv2.imshow("face blur", blur_img)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()